In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_excel("SP100_return.xlsx",index_col='Date',parse_dates=True)

In [3]:
df.head()

,AMT,COP,CVX,AXP,BAC,C,CB,CME,GS,JPM,...,TMO,UNH,AMZN,HD,LOW,MCD,NKE,SBUX,TGT,TJX
Date,,,,,,,,,,,,,,,,,,,,,
2007-06-27,0.022206,0.026145,0.017326,0.002358,0.006628,0.015845,-0.004513,0.020549,0.024320,0.016685,...,0.031496,0.001353,0.016792,0.019041,0.003240,-0.005927,0.030739,0.017758,0.009618,0.028079
2007-06-28,0.000654,-0.007271,0.000119,-0.004419,-0.003862,-0.003659,-0.006281,0.004153,-0.001186,-0.005399,...,-0.004642,0.010810,0.008082,-0.005047,-0.017265,-0.010955,-0.002375,-0.002267,0.001103,0.004593
2007-06-29,0.005251,0.017117,-0.003907,-0.001960,-0.009981,-0.012791,0.002611,-0.019107,-0.016688,-0.014768,...,0.004057,-0.020300,-0.009889,-0.008725,-0.002929,-0.001812,-0.001492,-0.012114,-0.001248,-0.008891
2007-07-02,0.021480,0.017496,0.009938,0.010241,0.005082,0.005336,0.008172,-0.003010,0.009246,0.012969,...,0.009049,0.009202,0.012580,0.000254,-0.003249,0.009388,-0.001368,-0.015622,0.008715,-0.001862
2007-07-06,0.017532,-0.002464,0.007031,-0.000106,-0.004873,-0.000870,-0.001978,0.049970,0.012025,0.003782,...,-0.004655,0.012623,0.003414,0.015115,0.010015,0.005651,0.004135,0.005477,0.057232,0.008365


In [4]:
returns=df*100

In [5]:
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.4/916.4 kB 10.7 MB/s eta 0:00:00


In [6]:
from arch import arch_model

In [7]:
df1=pd.read_excel("SP100_vol.xlsx", sheet_name='SP100 Original',index_col='Date',parse_dates=True)

In [8]:
df1.iloc[int(len(df1)*0.7)]

AMT     0.008684
COP     0.011713
CVX     0.008553
AXP     0.015038
BAC     0.009973
          ...   
MCD     0.004191
NKE     0.011698
SBUX    0.007688
TGT     0.010126
TJX     0.007017
Name: 2017-04-20 00:00:00, Length: 80, dtype: float64

In [9]:
forecasts1=np.zeros([int(len(df1)*0.3)-10,len(df.columns)])
test_size = int(len(df1)*0.3)
for i in range(len(df.columns)):
  am = arch_model(returns[df.columns[i]], vol="garch", p=1, o=0, q=1, dist="Normal")
  res = am.fit(update_freq=5,last_obs="2017-04-20")
  rolling_predictions = []
  for j in range(int(test_size/22)):
    train = returns[:-(test_size-j*22)]
    model = arch_model(train[df.columns[i]], p=1, o=0, q=1)
    model_fit = model.fit(disp='off')
    pred = model_fit.forecast(horizon=22)
    rolling_predictions.append(np.sqrt(pred.variance.values[-1,:]))
  rolling_predictions=np.sqrt(rolling_predictions)**2
  rolling_predictions=rolling_predictions.reshape(forecasts1.shape[0])
  forecasts1[:,i]=rolling_predictions/100

Iteration:      5,   Func. Count:     37,   Neg. LLF: 4098.551324373292
Iteration:     10,   Func. Count:     68,   Neg. LLF: 4046.2986831153808
Iteration:     15,   Func. Count:     93,   Neg. LLF: 4046.2968060279672
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4046.2968060279672
            Iterations: 16
            Function evaluations: 97
            Gradient evaluations: 16


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      5,   Func. Count:     34,   Neg. LLF: 4228.341052252861
Iteration:     10,   Func. Count:     64,   Neg. LLF: 4212.0769917771995
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4212.076323550774
            Iterations: 14
            Function evaluations: 84
            Gradient evaluations: 14


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      5,   Func. Count:     34,   Neg. LLF: 3675.214410051748
Iteration:     10,   Func. Count:     62,   Neg. LLF: 3670.6484708061225
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3670.6484708061225
            Iterations: 11
            Function evaluations: 66
            Gradient evaluations: 11


...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      5,   Func. Count:     34,   Neg. LLF: 218000.19874418777
Iteration:     10,   Func. Count:     64,   Neg. LLF: 3987.723545959655
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3987.4731659622666
            Iterations: 13
            Function evaluations: 79
            Gradient evaluations: 13


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      5,   Func. Count:     38,   Neg. LLF: 4077.4855698263227
Iteration:     10,   Func. Count:     67,   Neg. LLF: 4056.8462629649266
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4056.8462523465014
            Iterations: 13
            Function evaluations: 81
            Gradient evaluations: 13


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

In [10]:
df2=df1.iloc[int(len(df1)*0.7):len(df1)-11]

In [11]:
for i in range(len(df2.columns)):
  df2[df2.columns[i]]=forecasts1[:,i]

<ipython-input-11-ae522ed45476>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[df2.columns[i]]=forecasts1[:,i]


In [12]:
df2.to_excel("GARCH(1,1)-Norm forecasts.xlsx")